In [3]:
import numpy as np
import pandas as pd
import os
import glob

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib qt

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Slider, Select
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.io import export_png
from bokeh.models import DatetimeTickFormatter

In [6]:
import sys
sys.path.append("../")

processed_data_path = "/Users/augenpro/Documents/Empatica/data_sara/data/GGIR_input/"

start_end_sleep = np.load(processed_data_path + "SPT_window_GGIR.npy", allow_pickle=True)
ppg_df = pd.read_parquet(processed_data_path + "ppg.parquet")
temp_df = pd.read_parquet(processed_data_path + "temp.parquet")
acc_df = pd.read_parquet(processed_data_path + "acc.parquet")

In [7]:
# I need to divide it into portions when the device was 

t_empty_end = acc_df.index[acc_df.index.to_series().diff().dt.total_seconds() > 1]
t_empty_start = acc_df.index[np.where(acc_df.index.to_series().diff().dt.total_seconds() > 1)[0]-1]
t_empty = pd.DataFrame({"start": t_empty_start, "end": t_empty_end})

good_portions = pd.DataFrame(columns=["start", "end"])
good_portions["start"] = t_empty["end"].iloc[:-1].reset_index(drop=True)
good_portions["end"] = t_empty["start"].iloc[1:].reset_index(drop=True)
start_first_empty = t_empty["start"].iloc[0]
end_last_empty = t_empty["end"].iloc[-1]

# Segment the data into portions when the device was not in empty and perform nonwear detection
acc_df_portions = [acc_df[:start_first_empty]]
ppg_df_portions = [ppg_df[:start_first_empty]]

for i, row in good_portions.iterrows():

    if row["end"] - row["start"] < pd.Timedelta("10 min"): # if the portion is less than 10 minutes, skip it
        continue

    acc_df_portions.append(acc_df[row["start"]:row["end"]])
    ppg_df_portions.append(ppg_df[row["start"]:row["end"]])

acc_df_portions.append(acc_df[end_last_empty:])
ppg_df_portions.append(ppg_df[end_last_empty:])

In [8]:
from heart_rate.beliefppg.inference.inference import infer_hr

ModuleNotFoundError: No module named 'beliefppg'

In [54]:
for i, (acc_df_portion, ppg_df_portion) in enumerate(zip(acc_df_portions, ppg_df_portions)):
    plt.figure(figsize=(10, 5))
    plt.plot(ppg_df_portion.resample("0.1s").mean(), label=f"portion {i}")
    plt.plot(acc_df_portion["x"].resample("0.1s").mean(), label=f"portion {i}")

In [56]:
hr_all = []
time_hr_all = []
for acc, ppg in zip(acc_df_portions, ppg_df_portions): ##### Each portion has the same shape between ACC and PPG lesgooo
   time = acc.index # same as ppg.index
   hr, idxs = infer_hr(ppg=ppg.values.reshape(-1,1), ppg_freq=64, acc=acc.values, acc_freq=64)
   hr_all.append(hr)
   time_hr_all.append(time[idxs])

660/660 [==============================] - 59s 89ms/step


2025-02-13 11:49:30.038977: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


267/267 [==============================] - 21s 75ms/step


In [57]:
hr_belief = np.concatenate(hr_all)
t_hr_belief = np.concatenate(time_hr_all)

hr_belief.shape, t_hr_belief.shape

((233108,), (233108,))

In [59]:
hr_belief_df = pd.Series(hr_belief, index=t_hr_belief)
plt.figure()
plt.plot(hr_belief_df)

In [60]:
hr_belief_df.to_pickle(parquet_path + "hr_belief.pkl")